In [1]:
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
src = pathlib.Path().cwd().parent.parent / 'data' / 'processed'
ref = pathlib.Path().cwd().parent.parent / 'reference'

In [3]:
ct = pd.read_csv(src / 'ct-from-pcr-data' / 'ct.csv', index_col='date_sample')
# ct_by_awh = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-awh.csv')
ct_by_adh = pd.read_csv(src / 'ct-by-group-from-pcr-data' / 'ct-by-adh.csv')
# ct_by_sex = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-sex.csv')
# ct_by_ag = pd.read_csv(src / 'ct-by-group-from-test-data' / 'ct-by-ag.csv')
addiv = pd.read_csv(ref / 'addiv.csv')[['id_addiv', 'name_addiv_2']]
pop = pd.read_csv(ref / 'pop_1.csv')[['id_addiv', 'pop']]

In [4]:
ct

,no_test,no_positive,no_positive_group_sample,no_test_corrected,pct_positive,no_test_rollsum,no_positive_rollsum,pct_positive_per7d,ct,situational_level
date_sample,,,,,,,,,,
2020-07-04,2.0,0.0,0.0,2.0,0.000000,NaN,NaN,0.000000,1,2
2020-07-05,46.0,4.0,0.0,46.0,0.086957,NaN,NaN,0.000000,1,2
2020-07-06,1.0,0.0,0.0,1.0,0.000000,NaN,NaN,0.000000,1,2
2020-07-07,281.0,21.0,0.0,281.0,0.074733,NaN,NaN,0.000000,1,2
2020-07-08,0.0,0.0,0.0,0.0,0.000000,NaN,NaN,0.000000,1,2
...,...,...,...,...,...,...,...,...,...,...
2021-09-30,27008.0,2308.0,226.0,26782.0,0.086177,97501.0,23224.0,0.238192,4,3
2021-10-01,21453.0,2165.0,179.0,21274.0,0.101767,105789.0,21161.0,0.200030,4,3
2021-10-02,8590.0,1877.0,128.0,8462.0,0.221815,105151.0,19544.0,0.185866,3,3


In [5]:
tphcm = {
    'addr_dist_home':['THANH PHO HO CHI MINH'],
    'no_test_corrected': [ct[['no_test_corrected', 'no_positive']].sum()[0]],
    'no_positive': [ct[['no_test_corrected', 'no_positive']].sum()[1]]
}

df_tphcm = pd.DataFrame(tphcm)

df = (
    ct_by_adh[['date_sample', 'addr_dist_home', 'no_test_corrected', 'no_positive']]
#     .query('date_sample == "2021-08-05"')
    .groupby('addr_dist_home')[['no_test_corrected', 'no_positive']]
    .sum()
#     .to_frame(name='no_test_cumsum')
    .reset_index()
    .append(df_tphcm)
    .sort_values('no_test_corrected')
    .merge(
        addiv,
        how = 'left',
        left_on = 'addr_dist_home',
        right_on = 'name_addiv_2'
    )
    .merge(
        pop,
        how = 'left',
        left_on = 'id_addiv',
        right_on = 'id_addiv'
    )
    .drop(columns={'id_addiv', 'name_addiv_2'})
)

df['pct_positive'] = df['no_positive'] / df['no_test_corrected']

df['predicted_inc'] = df['pop'] * df['pct_positive']

df.to_csv(src / 'predicted-inc-from-test-data' / 'predicted-inc.csv', index=False)
df

,addr_dist_home,no_test_corrected,no_positive,pop,pct_positive,predicted_inc
0,HUYEN CAN GIO,26111.0,2371.0,71526,0.090805,6.494893e+03
1,QUAN 11,29491.0,8640.0,209867,0.292971,6.148489e+04
2,QUAN PHU NHUAN,35788.0,7157.0,163961,0.199983,3.278945e+04
3,QUAN 06,40293.0,19015.0,233561,0.471918,1.102217e+05
4,QUAN 01,41938.0,14971.0,142625,0.356979,5.091418e+04
5,HUYEN NHA BE,43966.0,22875.0,206837,0.520288,1.076149e+05
6,QUAN 02,44887.0,4702.0,180275,0.104752,1.888415e+04
7,QUAN 05,47390.0,15588.0,159073,0.328930,5.232391e+04
8,QUAN 04,50123.0,19028.0,175329,0.379626,6.655947e+04
9,QUAN TAN PHU,68881.0,24243.0,485348,0.351955,1.708206e+05


In [6]:
# df = (
#     ct_by_adh[['addr_dist_home', 'date_sample', 'no_test', 'no_positive']]
#     .groupby(['addr_dist_home', 'date_sample']).sum()
#     .groupby(level=0).cumsum().reset_index()
#     .merge(
#         addiv,
#         how = 'left',
#         left_on = 'addr_dist_home',
#         right_on = 'name_addiv_2'
#     )
#     .merge(
#         pop,
#         how = 'left',
#         left_on = 'id_addiv',
#         right_on = 'id_addiv'
#     )
#     .drop(columns={'id_addiv', 'name_addiv_2'})
# )

# df['pct_positive'] = df['no_positive'] / df['no_test']

# df['predicted_inc'] = df['pop'] * df['pct_positive']
# # foo[foo.addr_dist_home == 'HUYEN BINH CHANH'].tail(10)
# df